In [0]:
# Disclaimer: 
# The following code has been excessively commented for educational/demonstration purposes.
# In regular code, please follow a "clean code" approach to commenting instead.

In [0]:
import traceback
from pyspark.sql.functions import col
from configparser import NoSectionError

In [0]:
%run "../utility/util"

In [0]:
%run "../constants/constant"

In [0]:
class PipelineParameter:
  '''
  PipelineParameter holds all parameters passed from ADF pipeline
  '''
  def __init__(self, config_file, pipeline_runtime):
    self.config_file = config_file
    self.pipeline_runtime = pipeline_runtime
    
  def get_config_file(self):
    return self.config_file
  
  def get_pipeline_runtime(self): 
    return self.pipeline_runtime

In [0]:
def find_deleted_primary_keys(primary_keys, bronze_df, silver_df):
  '''
  Find non-matching primary keys between bronze and silver layer
  '''
  return silver_df.select([col(pk) for pk in primary_keys]) \
                 .subtract(bronze_df.select([col(pk) for pk in primary_keys]))

In [0]:
def read_records_from_bronze_layer(db_params, pipeline_runtime):
  '''
  This will read the data from parquet file in bronze layer.
  '''
  file_path = create_file_path(Constant.BRONZE_CONTAINER + "/primary_keys", db_params, pipeline_runtime)
  try:
    return file_reader("parquet", file_path)  
  except Exception:
    traceback.print_exc()
    raise Exception("Error while reading from parquet file " + file_path)

In [0]:
def read_records_from_silver_layer(db_params, primary_keys):
  '''
  This will read the data from delta table in silver layer
  '''
  try:
    return table_reader_delta(db_params.get_database(),  db_params.get_table(), primary_keys)
  except Exception:
    traceback.print_exc()
    raise Exception("Error while reading from Delta Table " + db_params.get_database() + "." \
                                                            +  db_params.get_table())

In [0]:
def get_join_condition(primary_keys):
  '''
  This will return the join condition to be used in delete query
  '''
  return "".join(["a." + pk + " = " + pk + " AND " for pk in primary_keys])[:-5]

In [0]:
def get_delete_query(db_params, primary_keys):
  '''
  Returns delete query which needs to be executed on delta table
  '''
  columns = ', '.join(primary_keys)
  return "DELETE FROM " + db_params.get_database() + "." + db_params.get_table() + \
                        " a WHERE EXISTS (SELECT " + columns + " FROM " \
                        + db_params.get_table() + "_diff WHERE " + get_join_condition(primary_keys) \
                        + ")"

In [0]:
def execute_delete_query(db_params, primary_keys):
  '''
  This will execute the delete query on delta table
  '''
  try:
    spark.sql(get_delete_query(db_params, primary_keys))
    spark.catalog.refreshTable(db_params.get_database() + "." + db_params.get_table())
  except Exception:
    traceback.print_exc()
    raise Exception("Error while executing delete query on delta table. Query = " + query)

In [0]:
def handle_silver_layer_deletions(server, tables, pipeline_params):
  '''
  This function performs the record deletion from delta tables
  '''
  password = dbutils.secrets.get(scope = "toromont-kv-secret", key = "datascienceazure")
  for table in tables:
    db_params = get_database_parameters(server, 
                                        table, 
                                        Constant.EXPECTED_TABLE_NAMESPACE_PARTS, 
                                        "datascienceazure", 
                                        password)
    primary_keys = get_primary_keys(pipeline_params.get_config_file(), "call_center_tables") \
                              .get(db_params.get_table())
    if len(primary_keys) == 0:
      raise Exception("primary_keys not present for table " + db_params.get_table())

    bronze_df = read_records_from_bronze_layer(db_params, pipeline_params.get_pipeline_runtime())
    silver_df = read_records_from_silver_layer(db_params, primary_keys)
    find_deleted_primary_keys(primary_keys, bronze_df, silver_df) \
                        .createOrReplaceTempView(db_params.get_table() + Constant.DELETE_TABLE_SUFFIX)  
    execute_delete_query(db_params, primary_keys)

In [0]:
def create_pipeline_parameters():
  # Read widget values sent from ADF pipeline
  config_file = get_stripped_param("config_file")
  pipeline_runtime = get_stripped_param("pipeline_runtime")
  
  return PipelineParameter(config_file, pipeline_runtime) 

In [0]:
def main():
  # Setting spark properties. Partition specifies no. of partition to create after shuffle operation.
  # This also indicates the no of parallel tasks that spark run to perform operation.
  set_spark_properties(Constant.SPARK_SHUFFLE_PARTITION)
  pipeline_params = create_pipeline_parameters()
  config_file = pipeline_params.get_config_file()

  try:
    initial_load = read_config(config_file, "call_center_tables", "initial_load")
    for server, tables in initial_load.items():
      handle_silver_layer_deletions(server, tables, pipeline_params)

  except FileNotFoundError:
    traceback.print_exc()
    raise Exception("Config file " + config_file + " does not exist.")
  except NoSectionError:
    traceback.print_exc()
    raise Exception("Section initial_load does not exist in " + config_file)

In [0]:
main()

Traceback (most recent call last):
 File "<command-2639850298033907>", line 7, in read_records_from_bronze_layer
 return file_reader("parquet", file_path)
 File "<command-4043258465741106>", line 6, in file_reader
 return spark.read.format("parquet").load(file_path)
 File "/databricks/spark/python/pyspark/sql/readwriter.py", line 178, in load
 return self._df(self._jreader.load(path))
 File "/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1305, in __call__
 answer, self.gateway_client, self.target_id, self.name)
 File "/databricks/spark/python/pyspark/sql/utils.py", line 133, in deco
 raise_from(converted)
 File "<string>", line 3, in raise_from
pyspark.sql.utils.AnalysisException: Path does not exist: dbfs:/mnt/bronze/primary_keys/dwdb/ms_f_invoiceheader_history/2020_11_13/2020_11_13-20_49_11;

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2639850298033907> in read_records_from_bronze_layer (db_params, pipeline_runtime) 
 6 try : 
 ----> 7 return file_reader ( "parquet" , file_path ) 
 8 except Exception : 

 <command-4043258465741106> in file_reader (source, file_path, header, schema, multiline) 
 5 if source . lower ( ) == "parquet" : 
 ----> 6 return spark . read . format ( "parquet" ) . load ( file_path ) 
 7 elif source . lower ( ) == "csv" : 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 177 if isinstance ( path , basestring ) : 
 --> 178 return self . _df ( self . _jreader . load ( path ) ) 
 179 elif path is not None : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : Path does not exist: dbfs:/mnt/bronze/primary_keys/dwdb/ms_f_invoiceheader_history/2020_11_13/2020_11_13-20_49_11;

During handling of the above exception, another exception occurred:

 Exception Traceback (most recent call last)
 <command-2639850298033915> in <module> 
 ----> 1 main ( ) 

 <command-2639850298033914> in main () 
 9 initial_load = read_config ( config_file , "call_center_tables" , "initial_load" ) 
 10 for server , tables in initial_load . items ( ) : 
 ---> 11 handle_silver_layer_deletions ( server , tables , pipeline_params ) 
 12 
 13 except FileNotFoundError : 

 <command-2639850298033912> in handle_silver_layer_deletions (server, tables, pipeline_params) 
 15 raise Exception ( "primary_keys not present for table " + db_params . get_table ( ) ) 
 16 
 ---> 17 bronze_df = read_records_from_bronze_layer ( db_params , pipeline_params . get_pipeline_runtime ( ) ) 
 18 silver_df = read_records_from_silver_layer ( db_params , primary_keys ) 
 19 find_deleted_primary_keys ( primary_keys , bronze_df , silver_df ) \ 

 <command-2639850298033907> in read_records_from_bronze_layer (db_params, pipeline_runtime) 
 8 except Exception : 
 9 traceback . print_exc ( ) 
 ---> 10 raise Exception ( "Error while reading from parquet file " + file_path ) 

 Exception : Error while reading from parquet file /mnt/bronze/primary_keys/dwdb/ms_f_invoiceheader_history/2020_11_13/2020_11_13-20_49_11